In [1]:
import IPython.display
import ipyvolume.pylab as p3d
import matplotlib.cm
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import tiledb

In [3]:
tiledb.libtiledb.version()

(1, 2, 1)

In [4]:
ctx = tiledb.Ctx()

In [5]:
arr = tiledb.SparseArray.load(ctx, "data/nyu_dublin_lidar_decimated")

In [6]:
arr.domain

Domain(Dim(name='x', domain=(0, 4200000000), tile=100000, dtype=uint32),
       Dim(name='y', domain=(0, 4200000000), tile=100000, dtype=uint32),
       Dim(name='z', domain=(0, 4200000000), tile=100000, dtype=uint32))

In [7]:
data = arr[:]
data["coords"][:10]

array([(2145882748, 2148575367, 2147550830),
       (2145883036, 2148575274, 2147550861),
       (2145883252, 2148575244, 2147550495),
       (2145883391, 2148575121, 2147550879),
       (2145883483, 2148575056, 2147551338),
       (2145883570, 2148574998, 2147551523),
       (2145883668, 2148574969, 2147551376),
       (2145883762, 2148574748, 2147553648),
       (2145883851, 2148574966, 2147551219),
       (2145883920, 2148574979, 2147550606)],
      dtype=[('x', '<u4'), ('y', '<u4'), ('z', '<u4')])

In [8]:
X = data["coords"]['x'][::10].astype(float)
Y = data["coords"]['y'][::10].astype(float)
Z = data["coords"]['z'][::10].astype(float) * 2 # need to give some height to this state
Xadj = X - X.min()
Yadj = Y - Y.min()
Zadj = Z - Z.min()

In [10]:
fig = p3d.figure(width=1000)
fig.xlabel='Y'
fig.ylabel='Z'
fig.zlabel='X'
all_points = p3d.scatter(Yadj, Zadj, Xadj, color='red', size=0.08)
p3d.squarelim()
p3d.show()

VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white', dtype='<U5'), size=array(0.08), size_selected=array(2.6), x=array([1412306., 1411681., 1411586., ...,  323906.,  323879.,  324791.]), y=array([106900., 111138., 108274., ..., 132092., 132070., 132054.]), z=array([  17926.,   19127.,   19924., ..., 1666534., 1667244., 1668723.]))], style={'axes': {'visible': True, 'label': {'color': 'black'}, 'ticklabel': {'color': 'black'}, 'color': 'black'}, 'box': {'visible': True}, 'background-color': 'white'}, tf=None, width=1000, xlabel='Y', xlim=[0.0, 1937146.0], ylabel='Z', ylim=[-858793.0, 1078353.0], zlabel='X', zlim=[-134211.5, 1802934.5]),))

In [11]:
# Select out the interesting columsns
df = pd.DataFrame(data=data, columns=["coords", "intensity", "classification", "point_source_id"])
df.head(5)

,coords,intensity,classification,point_source_id
0,"(2145882748, 2148575367, 2147550830)",79,2,0
1,"(2145883036, 2148575274, 2147550861)",79,4,0
2,"(2145883252, 2148575244, 2147550495)",61,4,0
3,"(2145883391, 2148575121, 2147550879)",121,2,0
4,"(2145883483, 2148575056, 2147551338)",97,2,0


In [12]:
# plot the classification numbers with different colors
coords1 = data["coords"][(data["classification"] & 0xf) == 2]
coords2 = data["coords"][(data["classification"] & 0xf) == 4]
print("N points: Class 1: {}".format(len(coords1)))
print("N points: Class 2: {}".format(len(coords2)))

N points: Class 1: 2716610
N points: Class 2: 1309844


In [13]:
# Plot the classification points
X1 = coords1['x'][::10].astype(float)
Y1 = coords1['y'][::10].astype(float)
Z1 = coords1['z'][::10].astype(float)
min_ground = Z1.min()
X1 = X1 - X1.min()
Y1 = Y1 - Y1.min()
Z1 = Z1 - min_ground
groundarr = np.column_stack((X1,Y1))
groundtri = Delaunay(groundarr)

X2 = coords2['x'][::10].astype(float)
Y2 = coords2['y'][::10].astype(float)
Z2 = coords2['z'][::10].astype(float)
X2 = X2 - X2.min()
Y2 = Y2 - Y2.min()
Z2 = Z2 - min_ground
vegarr = np.column_stack((X2,Y2))
vegtri = Delaunay(vegarr)

class_fig = p3d.figure(width=1000)
class_fig.xlabel='Y'
class_fig.ylabel='Z'
class_fig.zlabel='X'
p3d.plot_trisurf(Y1, Z1, X1, triangles=groundtri.simplices, color='grey')
#p3d.plot_trisurf(Y2, Z2, X2, triangles=vegtri.simplices, color='green')
#c1_points = p3d.scatter(Y1, Z1, X1, color='grey', size=0.08)
c2_points = p3d.scatter(Y2, Z2, X2, color='green', size=0.03)
#class_fig.scatters.append(c1_points)
class_fig.scatters.append(c2_points)
p3d.squarelim()
p3d.show()

VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], meshes=[Mesh(color=array('grey', dtype='<U4'), texture=None, triangles=array([[125191, 199470,  33096],
       [250232, 234184, 234134],
       [ 16068,      0,  33271],
       ...,
       [153215, 153203, 153152],
       [153215, 153221, 153204],
       [153203, 153215, 153204]], dtype=uint32), x=array([1413420., 1412642., 1412234., ...,  328082.,  324831.,  324953.]), y=array([30366., 30395., 30713., ..., 42974., 42911., 42924.]), z=array([  18208.,   20880.,   22035., ..., 1666235., 1667000., 1668418.]))], scatters=[Scatter(color=array('green', dtype='<U5'), color_selected=array('white', dtype='<U5'), size=array(0.03), size_selected=array(2.6), x=array([1408954., 1408299., 1407983., ...,  313966.,  314733.,  319812.]), y=array([30397., 32469., 33626., ..., 47004., 48512., 42854.]), z=array([  17210.,   18635.,   19398., ..., 1662395., 1662856., 1664015.])), Scatter(color=array('green', dtype='<U5'), color_selected=array('white', dtype='<U5'), size=array(0.03), size_selected=array(2.6), x=array([1408954., 1408299., 1407983., ...,  313966.,  314733.,  319812.]), y=array([30397., 32469., 33626., ..., 47004., 48512., 42854.]), z=array([  17210.,   18635.,   19398., ..., 1662395., 1662856., 1664015.]))], style={'axes': {'visible': True, 'label': {'color': 'black'}, 'ticklabel': {'color': 'black'}, 'color': 'black'}, 'box': {'visible': True}, 'background-color': 'white'}, tf=None, width=1000, xlabel='Y', xlim=[0.0, 1933231.0], ylabel='Z', ylim=[-797055.0, 1136176.0], zlabel='X', zlim=[-132406.5, 1800824.5]),))